In [1]:
import torch
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../..")
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
from src.models import metrics
import gin

/home/azureuser/.cache/pypoetry/virtualenvs/poetry_add_pandas-Tn0MmM9i-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-12 18:04:44.012485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 18:04:44.220846: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 18:04:44.220880: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ign

In [2]:
gin.parse_config_file("model.gin")
train_dataloader, test_dataloader = make_dataset.get_MNIST()
accuracy = metrics.Accuracy()
# print(gin.config_str())

In [3]:

def hand_hypertuning(
    learning_rates,
    kernel_sizes,
    filters1,
    filters2,
    units1,
    units2,
    model_dic,
    epoch = 3,
    num_classes = 10,
    log_dir = "../../models/test/"):
  
    # make list if not list:
    def make_ls(x):
        return x if type(x) == type([]) else [x]

    # maak van loopers list if not:
    learning_rates = make_ls(learning_rates) 
    kernel_sizes = make_ls(kernel_sizes) 
    filters1 = make_ls(filters1) 
    filters2 = make_ls(filters2) 
    units1 = make_ls(units1) 
    units2 = make_ls(units2)

    #model naam en model object vullen:
    
    model_name = model_dic[0]
    
    
    #gin_config resetten en vullen:
    gin.clear_config()
    gin.parse_config_file("model.gin")
    
    gin.bind_parameter("trainloop.log_dir",log_dir)
    gin.bind_parameter("trainloop.epochs", epoch)
    gin.bind_parameter(model_name + '.num_classes', num_classes)
    # gin.bind_parameter(model_name + '.kernel_size', kernel_size)

    for fl1 in filters1:
        for fl2 in filters2:
            if fl2 <= fl1:
                for ut1 in units1:
                    for ut2 in units2:
                        if ut2 <= ut1:
                            for lr in learning_rates:
                                for kerz in kernel_sizes:
                                
                                    gin.bind_parameter(model_name + '.filter1', fl1)
                                    gin.bind_parameter(model_name + '.filter2', fl2)
                                    gin.bind_parameter(model_name + '.unit1', ut1)
                                    gin.bind_parameter(model_name + '.unit2', ut2)
                                    gin.bind_parameter(model_name + '.kernel_size', kerz)
                                    gin.bind_parameter("trainloop.learning_rate", lr)
                                    # print(gin.config_str())

                                    model = model_dic[1]

                                    model =  train_model.trainloop(
                                        model=model(),
                                        metrics=[accuracy],
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        train_steps=len(train_dataloader),
                                        eval_steps=150,
                                    )
    
    return None

model_dic = {
    'CNN':{0:'CNN',1:imagemodels.CNN}
    ,'CNN_150': {0:'CNN_150',1:imagemodels.CNN_150}
    }
    


Als eerste wordt de learning rate getest:

In [4]:

# hand_hypertuning(
#     model_dic = model_dic['CNN']
#     ,learning_rates = [0.01, 0.001, 0.0001]
#     ,kernel_sizes = 3
#     ,filters1 = 128
#     ,filters2 = 128
#     ,units1 = 128
#     ,units2 = 64
#     ,log_dir = "../../models/learning_rate/")

Nu we weten dat de optimale learning rate ligt op ongeveer 0.001 kan verder naar andere optimuns worden gezocht.
Als eerste wordt kernelsize getest:

Kernal; size is optimaal op 2 omdat filters en units de onformatie aan elkaar doorgeven, hebben filter zise en unitr een invloed op elkaar. De units nemen het over op een platten 128. dit geeft beperkingen deze zijn meegenomen:

In [5]:
# fil_range_e3 = [16,32,64,128]
# fil_range_e10 = [32,64]

# hand_hypertuning(
#     model_dic = model_dic['CNN']
#     ,learning_rates = 0.001
#     ,kernel_sizes = 2
#     ,filters1 = fil_range_e10
#     ,filters2 = fil_range_e10
#     ,units1 = 128
#     ,units2 = 64
#     ,log_dir = "../../models/filters/"
#     ,epoch = 10
#     )

In [6]:
unit_range_e3 = [16,32,64,128]
unit_range_e10 = [32,64,128]

# hand_hypertuning(
#     model_dic = model_dic['CNN']
#     ,learning_rates = 0.001
#     ,kernel_sizes = 2
#     ,filters1 = 64
#     ,filters2 = 64
#     ,units1 = 128
#     ,units2 = 128
#     ,log_dir = "../../models/units/"
#     ,epoch = 3
#     )

In [7]:
fil_range_e3 = [16,32,64,128]
# fil_range_e10 = [32,64]

hand_hypertuning(
    model_dic = model_dic['CNN_150']
    ,learning_rates = 0.001
    ,kernel_sizes = 2
    ,filters1 = fil_range_e3
    ,filters2 = fil_range_e3
    ,units1 = 128
    ,units2 = 128
    ,log_dir = "../../models/filters_cnn150/"
    ,epoch = 3
    )

2022-12-12 18:04:50.019 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters_cnn150/20221212-1804
100%|██████████| 1875/1875 [00:31<00:00, 58.85it/s]
2022-12-12 18:05:23.458 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.7634 test 0.5661 metric ['0.7758']
100%|██████████| 1875/1875 [00:30<00:00, 61.04it/s]
2022-12-12 18:05:55.181 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.4487 test 0.4155 metric ['0.8321']
100%|██████████| 1875/1875 [00:30<00:00, 62.15it/s]
2022-12-12 18:06:26.383 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.3783 test 0.4050 metric ['0.8473']
100%|██████████| 3/3 [01:35<00:00, 31.95s/it]
2022-12-12 18:06:26.390 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/filters_cnn150/20221212-1806
100%|██████████| 1875/1875 [00:37<00:00, 49.65it/s]
2022-12-12 18:07:05.719 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.6780 

KeyboardInterrupt: 

In [ ]:
%tensorboard